Imports

In [1155]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
from plotting import plot_array
from soundGen import play
from pointsFrequency import signal_to_hertz
from filterAudio import low_pass_Filter, high_pass_Filter,\
     dirac_comb_discrete, weierstrassFunc
import torch
from torch_pitch_shift import pitch_shift
from numpy.random import uniform


Set universal variables above functions and declare them 

In [1156]:
Fs = 44100
T = 1/(Fs) 
N = Fs
amplitude = 2
frequency = [400, 200, 100]
t = 1
omega = 2 * np.pi


Then create code for signal

In [1157]:
t_vec = np.arange(N)*T*t 
ysum = 0
sine_add = np.sin(np.linspace(-16 * np.pi,16 * np.pi,len(t_vec)))/10
omega = 2 * np.pi* sine_add
for i in range(len(frequency)):
    ysum  += amplitude * np.sin(omega * frequency[i] * t_vec)


Normalize the signal


In [1158]:
normalized_y = ysum/ np.max(ysum)

weier = weierstrassFunc(normalized_y, a = 0.5)
plt.plot(t_vec, weier + normalized_y)
plt.show()

TypeError: weierstrassFunc() missing 1 required positional argument: 'b'